In [ ]:
import cv2
import os
import numpy as np
import statistics
import re
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')

# Definir rutas base
carpeta_base = "/content/drive/MyDrive/disco_diffusion"
categorias = ["art", "faces", "landscapes"]

# Inicializar variables para estadísticas
todas_distancias = []
todas_coincidencias = []
promedio_caracteristicas = []
resultados = []

# Inicializar ORB
orb = cv2.ORB_create()
matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Función para calcular distancia Hamming entre descriptores
def calcular_distancia(matches):
    distancias = [match.distance for match in matches]
    return np.mean(distancias) if distancias else 0

# Procesar cada categoría
for categoria in categorias:
    carpeta_eng = os.path.join(carpeta_base, categoria, "english")
    carpeta_esp = os.path.join(carpeta_base, categoria, "spanish")
    carpeta_guardado = os.path.join("/content", "PROCESADAS - ORB", categoria)
    os.makedirs(carpeta_guardado, exist_ok=True)

    imagenes_eng = {img: os.path.join(carpeta_eng, img) for img in os.listdir(carpeta_eng) if img.endswith(('.png', '.jpg', '.jpeg'))}
    imagenes_esp = {img: os.path.join(carpeta_esp, img) for img in os.listdir(carpeta_esp) if img.endswith(('.png', '.jpg', '.jpeg'))}

    for eng_img, eng_path in imagenes_eng.items():
        num = re.search(r'\d+', eng_img).group()  # Extraer número de la imagen
        esp_img = f"spanish_image_{num}.png"

        if esp_img in imagenes_esp:
            esp_path = imagenes_esp[esp_img]

            # Leer imágenes en escala de grises
            img1 = cv2.imread(eng_path, cv2.IMREAD_GRAYSCALE)
            img2 = cv2.imread(esp_path, cv2.IMREAD_GRAYSCALE)

            # Extraer características ORB
            kp1, des1 = orb.detectAndCompute(img1, None)
            kp2, des2 = orb.detectAndCompute(img2, None)

            # Hacer matching
            if des1 is not None and des2 is not None:
                matches = matcher.match(des1, des2)
                distancia_promedio = calcular_distancia(matches)
                cantidad_matches = len(matches)

                todas_distancias.append((categoria, distancia_promedio))
                todas_coincidencias.append(cantidad_matches)
                promedio_caracteristicas.append((len(kp1) + len(kp2)) / 2)

                # Guardar resultados en lista
                resultados.append([categoria, num, distancia_promedio, cantidad_matches])

                # Dibujar coincidencias
                img_matches = cv2.drawMatches(img1, kp1, img2, kp2, matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

                # Guardar imagen procesada
                nombre_guardado = os.path.join(carpeta_guardado, f"procesada_{num}.png")
                cv2.imwrite(nombre_guardado, img_matches)

                 # Mostrar imagen procesada
                plt.figure(figsize=(10, 5))
                plt.imshow(cv2.cvtColor(img_matches, cv2.COLOR_BGR2RGB))
                plt.title(f"Imagen {num} - {categoria}")
                plt.axis("off")
                plt.show()

                # Imprimir resultados por imagen
                print(f"Imagen {num} - {categoria}:")
                print(f"  - Distancia promedio: {distancia_promedio:.2f}")
                print(f"  - Matches encontrados: {cantidad_matches}\n")

# Guardar resultados en CSV
df_resultados = pd.DataFrame(resultados, columns=["Categoria", "Numero", "Distancia Promedio", "Matches"])
df_resultados.to_csv("/content/resultados_orb.csv", index=False)

# Calcular estadísticas finales
if todas_distancias:
    print("\nESTADÍSTICAS FINALES:")
    print(f"  - Distancia máxima: {max([d[1] for d in todas_distancias]):.2f}")
    print(f"  - Distancia mínima: {min([d[1] for d in todas_distancias]):.2f}")
    print(f"  - Distancia media: {statistics.mean([d[1] for d in todas_distancias]):.2f}")
    print(f"  - Desviación estándar: {statistics.stdev([d[1] for d in todas_distancias]):.2f}")
    print(f"  - Máximo matches: {max(todas_coincidencias)}")
    print(f"  - Mínimo matches: {min(todas_coincidencias)}")
    print(f"  - Promedio de matches: {statistics.mean(todas_coincidencias):.2f}")
    print(f"  - Promedio de características detectadas: {statistics.mean(promedio_caracteristicas):.2f}")


Mounted at /content/drive
Imagen 2 - art:
  - Distancia promedio: 65.07
  - Matches encontrados: 150

Imagen 3 - art:
  - Distancia promedio: 58.94
  - Matches encontrados: 170

Imagen 4 - art:
  - Distancia promedio: 60.40
  - Matches encontrados: 141

Imagen 5 - art:
  - Distancia promedio: 64.11
  - Matches encontrados: 148

Imagen 6 - art:
  - Distancia promedio: 64.50
  - Matches encontrados: 139

Imagen 7 - art:
  - Distancia promedio: 63.12
  - Matches encontrados: 130

Imagen 8 - art:
  - Distancia promedio: 66.09
  - Matches encontrados: 169

Imagen 9 - art:
  - Distancia promedio: 61.71
  - Matches encontrados: 163

Imagen 10 - art:
  - Distancia promedio: 62.34
  - Matches encontrados: 148

Imagen 11 - art:
  - Distancia promedio: 63.89
  - Matches encontrados: 153

Imagen 12 - art:
  - Distancia promedio: 65.25
  - Matches encontrados: 154

Imagen 13 - art:
  - Distancia promedio: 60.53
  - Matches encontrados: 142

Imagen 14 - art:
  - Distancia promedio: 61.77
  - Matches

In [ ]:
def graficar_boxplot():
    plt.figure(figsize=(10, 6))

    # Organizar las distancias en listas por categoría
    distancias_por_categoria = {categoria: [] for categoria in categorias}

    for categoria, distancia in todas_distancias:
        distancias_por_categoria[categoria].append(distancia)

    # Crear el boxplot con colores
    colors = ['lightblue', 'lightgreen', 'lightcoral']
    box = plt.boxplot(distancias_por_categoria.values(), labels=distancias_por_categoria.keys(), patch_artist=True)

    for patch, color in zip(box['boxes'], colors):
        patch.set_facecolor(color)

    # Etiquetas y título
    plt.xlabel("Categoría")
    plt.ylabel("Distancia Promedio")
    plt.title("Distancias entre imágenes")

    # Mostrar la gráfica
    plt.grid(True)
    plt.show()

graficar_boxplot()


In [ ]:
# Código adicional para graficar la cantidad de matches
def graficar_matches():
    plt.figure(figsize=(10, 6))
    plt.plot([r[1] for r in resultados], [r[3] for r in resultados], marker='o', linestyle='-', color='b')
    plt.xlabel("Número de Imagen")
    plt.ylabel("Cantidad de Matches")
    plt.title("Cantidad de Matches por Imagen")
    plt.grid(True)
    plt.show()

graficar_matches()
